In [15]:
from seq2seq_dialogue import create_model
import json
from codecs import getreader
from os import path, makedirs
import numpy as np

from batch_generator import BatchGenerator, generate_sequences

In [4]:
with getreader('utf-8')(open('config.json')) as config_in:
    config = json.load(config_in)

In [8]:
MODEL_FILE = config['model_weights']
MODEL_DIR = path.dirname(MODEL_FILE)
if not path.exists(MODEL_DIR):
    makedirs(MODEL_DIR)
with getreader('utf-8')(open(config['vocabulary'])) as vocab_in:
    VOCAB = [line.strip() for line in vocab_in]
EMBEDDINGS = np.load(config['embeddings_matrix'])
BUCKETS = config['buckets']

In [9]:
model_1 = create_model(VOCAB, VOCAB, EMBEDDINGS, BUCKETS[1][0], BUCKETS[1][1])
model_1.save_weights(MODEL_FILE, overwrite=True)
del model_1

In [16]:
model_2 = create_model(VOCAB, VOCAB, EMBEDDINGS, BUCKETS[2][0], BUCKETS[2][1])
model_2.load_weights(MODEL_FILE)
train_batch_generator = BatchGenerator(
    config['train_set'],
    config['batch_size'],
    VOCAB,
    BUCKETS[2]
)

model_2.fit_generator(
    generate_sequences(train_batch_generator),
    nb_epoch=2,
    samples_per_epoch=32
)

Epoch 1/2
32/32 [==============================] - 30s - loss: 11.1481    
Epoch 2/2
32/32 [==============================] - 30s - loss: 11.1085    
